In [1]:
# @title Komórka 1: Instalacja i import bibliotek, pobieranie zasobów NLTK
# Zainstaluj/zaktualizuj potrzebne biblioteki
!pip install -q gensim pandas nltk scikit-learn matplotlib # Dodajemy gensim

import os
import pandas as pd
import numpy as np
import re
import time
import gc # Garbage Collector
import random

# NLTK do przetwarzania tekstu
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Gensim do modelowania tematów i spójności
import gensim
import gensim.corpora as corpora
from gensim.models import LsiModel, LdaMulticore, CoherenceModel

# Do obsługi ostrzeżeń
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning) # Ignoruj ostrzeżenia o przestarzałych funkcjach (częste w gensim)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# --- Konfiguracja ---
DATA_FILE_PATH = '../../data/dataset_combined.csv' # Ścieżka do pliku danych
OUTPUT_DIR = '../../results/topic_modeling_results_gensim' # Nowy katalog na wyniki z gensim
NUM_TOPICS_LIST = [75, 100] # Liczby tematów do analizy
TOP_N_WORDS = 10 # Ile słów zapisać dla każdego tematu

# --- Pobieranie zasobów NLTK ---
print("Pobieranie zasobów NLTK (punkt, stopwords, wordnet)...")
try:
    import ssl
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
    print("Zasoby NLTK pobrane.")
except Exception as e:
    print(f"Wystąpił błąd podczas pobierania zasobów NLTK: {e}")

# Utworzenie katalogu na wyniki, jeśli nie istnieje
os.makedirs(OUTPUT_DIR, exist_ok=True)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Pobieranie zasobów NLTK (punkt, stopwords, wordnet)...
Zasoby NLTK pobrane.


In [2]:
# @title Komórka 2: Wczytanie i wstępne przygotowanie danych (bez zmian)
print(f"Wczytywanie danych z pliku: {DATA_FILE_PATH}...")
start_time = time.time()
try:
    df = pd.read_csv(DATA_FILE_PATH, low_memory=False)
    print(f"Wczytano {len(df)} wierszy.")
    print(f"Czas wczytywania: {time.time() - start_time:.2f}s")

    print("\nDostępne kolumny:", df.columns.tolist())
    text_column = 'review'
    if text_column not in df.columns:
        raise ValueError(f"Nie znaleziono kolumny '{text_column}' w pliku CSV.")

    print(f"\nLiczba brakujących wartości w kolumnie '{text_column}': {df[text_column].isnull().sum()}")
    df.dropna(subset=[text_column], inplace=True)
    df[text_column] = df[text_column].astype(str)
    print(f"Liczba wierszy po usunięciu brakujących wartości: {len(df)}")

    # --- Opcjonalne próbkowanie dla szybszych testów ---
    # sample_size = 10000 # Mała próbka do testów
    # if len(df) > sample_size:
    #     print(f"\nZmniejszanie zbioru danych do {sample_size} próbek dla testów...")
    #     df = df.sample(n=sample_size, random_state=42)
    #     gc.collect()
    # ----------------------------------------------------

    texts = df[text_column].tolist()
    print(f"\nPrzykładowa recenzja (przed przetwarzaniem):\n{texts[0][:500]}...")

    del df # Usuń DataFrame, aby zwolnić pamięć
    gc.collect()

except FileNotFoundError:
    print(f"BŁĄD: Nie znaleziono pliku {DATA_FILE_PATH}.")
    texts = None
except ValueError as ve:
    print(f"BŁĄD: {ve}")
    texts = None
except Exception as e:
    print(f"Wystąpił nieoczekiwany błąd podczas wczytywania danych: {e}")
    texts = None

Wczytywanie danych z pliku: ../../data/dataset_combined.csv...
Wczytano 253073 wierszy.
Czas wczytywania: 4.05s

Dostępne kolumny: ['recommendationid', 'author', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access', 'primarily_steam_deck', 'appid', 'fetch_date', 'review_type', 'total_reviews', 'timestamp_dev_responded', 'developer_response']

Liczba brakujących wartości w kolumnie 'review': 407
Liczba wierszy po usunięciu brakujących wartości: 252666

Przykładowa recenzja (przed przetwarzaniem):
As we get older, playing games feels boring, even though there is something exciting in the game, at least 1-3 hours then we will feel bored again, but it's different with this game, once you play 1 hour you will definitely be attached and really want to spend the whole day playing this game. 

This game succeeded in making me feel like a

In [3]:
# @title Komórka 3: Preprocessing tekstu i tworzenie słownika/korpusu Gensim

# Inicjalizacja lematyzatora i listy stopwords
lemmatizer = WordNetLemmatizer()
custom_stop_words = {
    'game', 'games', 'play', 'played', 'playing', 'player', 'players',
    'steam', 'review', 'reviews', 'recommend', 'recommended', 'recommendation',
    'hour', 'hours', 'hr', 'hrs',
    'like', 'good', 'bad', 'great', 'best', 'worst', 'fun', 'nice', 'well',
    'really', 'much', 'many', 'lot', 'also', 'get', 'got', 'make', 'made',
    'time', 'people', 'would', 'one', 'even', 'go', 'going', 'want', 'say',
    'thing', 'think', 'know', 'see', 'come', 'back', 'use', 'still',
    'early', 'access', 'ea', 'dlc', 'buy', 'bought', 'purchase', 'price', 'worth',
    'pc', 'console', 'etc', 've', 're' # Dodano kilka typowych skrótów/wypełniaczy
}
stop_words = set(stopwords.words('english')).union(custom_stop_words)
print(f"Rozmiar słownika stopwords: {len(stop_words)}")

# ZMODYFIKOWANA funkcja preprocessingu ZWRACAJĄCA LISTĘ TOKENÓW
def preprocess_text_gensim(text):
    """
    Czyści tekst i zwraca listę tokenów (słów).
    """
    if not isinstance(text, str) or not text.strip():
        return []

    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-z\s]', '', text) # Usuń wszystko poza literami i spacjami
    tokens = word_tokenize(text)
    processed_tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]
    return processed_tokens # Zwraca listę tokenów

# Zastosowanie preprocessingu
if texts:
    print("\nRozpoczynanie preprocessingu tekstów dla Gensim...")
    start_time = time.time()
    processed_docs = [preprocess_text_gensim(doc) for doc in texts]
    print(f"Preprocessing zakończony. Czas: {time.time() - start_time:.2f}s")

    # Usuń puste listy tokenów, które mogły powstać
    processed_docs = [doc for doc in processed_docs if doc]
    print(f"Liczba dokumentów po preprocessingu i usunięciu pustych: {len(processed_docs)}")

    # Usuń oryginalne teksty, aby zwolnić pamięć
    del texts
    gc.collect()

    if processed_docs:
        print(f"\nPrzykładowy dokument (po preprocessingu, jako lista tokenów):\n{processed_docs[0][:20]}...")

        # --- Tworzenie słownika i korpusu Gensim ---
        print("\nTworzenie słownika Gensim...")
        dictionary = corpora.Dictionary(processed_docs)
        print(f"Rozmiar słownika przed filtrowaniem: {len(dictionary)}")

        # Filtrowanie ekstremalnych słów (podobnie do min_df, max_df, max_features)
        dictionary.filter_extremes(
            no_below=5,      # Ignoruj słowa występujące w mniej niż 5 dokumentach
            no_above=0.7,    # Ignoruj słowa występujące w więcej niż 70% dokumentów
            keep_n=10000     # Zachowaj max 10000 najczęstszych słów po filtrowaniu
        )
        dictionary.compactify() # Przypisz nowe, ciągłe ID po usunięciu słów
        print(f"Rozmiar słownika po filtrowaniu: {len(dictionary)}")

        print("\nTworzenie korpusu Bag-of-Words (BoW)...")
        # Tworzenie korpusu (lista list krotek: (id_slowa, liczba_wystapien))
        corpus_bow = [dictionary.doc2bow(doc) for doc in processed_docs]

        # Sprawdzenie, czy korpus nie jest pusty
        if not corpus_bow or not any(corpus_bow):
             print("BŁĄD: Korpus BoW jest pusty. Sprawdź wyniki preprocessingu.")
             # Ustaw zmienne na None, aby kolejne komórki nie próbowały ich użyć
             processed_docs = None
             dictionary = None
             corpus_bow = None
        else:
             print(f"Utworzono korpus BoW dla {len(corpus_bow)} dokumentów.")
             print(f"Przykładowy wpis w korpusie BoW (dla pierwszego dokumentu):\n{corpus_bow[0][:10]}...") # Pokaż początek

             # Utworzenie korpusu TF-IDF (potrzebny dla LSA)
             print("\nTworzenie modelu TF-IDF i korpusu TF-IDF...")
             tfidf_model = gensim.models.TfidfModel(corpus_bow, id2word=dictionary)
             corpus_tfidf = tfidf_model[corpus_bow]
             print(f"Utworzono korpus TF-IDF.")
             print(f"Przykładowy wpis w korpusie TF-IDF (dla pierwszego dokumentu):\n{corpus_tfidf[0][:10]}...")

    else:
        print("Lista 'processed_docs' jest pusta po preprocessingu.")
        dictionary = None
        corpus_bow = None
        corpus_tfidf = None

else:
    print("Brak tekstów wejściowych do przetworzenia.")
    processed_docs = None
    dictionary = None
    corpus_bow = None
    corpus_tfidf = None

gc.collect() # Zwolnij pamięć

Rozmiar słownika stopwords: 262

Rozpoczynanie preprocessingu tekstów dla Gensim...
Preprocessing zakończony. Czas: 97.87s
Liczba dokumentów po preprocessingu i usunięciu pustych: 243845

Przykładowy dokument (po preprocessingu, jako lista tokenów):
['older', 'feel', 'boring', 'though', 'something', 'exciting', 'least', 'feel', 'bored', 'different', 'definitely', 'attached', 'spend', 'whole', 'day', 'succeeded', 'making', 'feel', 'child', 'used']...

Tworzenie słownika Gensim...
Rozmiar słownika przed filtrowaniem: 105275
Rozmiar słownika po filtrowaniu: 10000

Tworzenie korpusu Bag-of-Words (BoW)...
Utworzono korpus BoW dla 243845 dokumentów.
Przykładowy wpis w korpusie BoW (dla pierwszego dokumentu):
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1)]...

Tworzenie modelu TF-IDF i korpusu TF-IDF...
Utworzono korpus TF-IDF.
Przykładowy wpis w korpusie TF-IDF (dla pierwszego dokumentu):
[(0, 0.23147332771606596), (1, 0.18586613368597454), (2, 0.117461564898

0

In [4]:
# @title Komórka 4: Funkcja do modelowania tematów Gensim (LSA/LDA) i obliczania spójności C_v

def run_gensim_topic_modeling(processed_docs, dictionary, corpus, method, n_topics, top_n_words, output_dir):
    """
    Wykonuje modelowanie tematów (LSA lub LDA) używając Gensim,
    oblicza spójność C_v i zapisuje wyniki do pliku CSV.

    Args:
        processed_docs (list of list of str): Lista dokumentów jako listy tokenów (potrzebna do CoherenceModel).
        dictionary (gensim.corpora.Dictionary): Słownik Gensim.
        corpus (iterable): Korpus Gensim (BoW dla LDA, TF-IDF dla LSA).
        method (str): 'lsa' lub 'lda'.
        n_topics (int): Liczba tematów do wyodrębnienia.
        top_n_words (int): Liczba najważniejszych słów do zapisania dla każdego tematu.
        output_dir (str): Katalog do zapisu pliku CSV.

    Returns:
        None
    """
    if not processed_docs or not dictionary or not corpus:
        print(f"BŁĄD: Brak danych wejściowych (processed_docs, dictionary lub corpus) dla metody {method.upper()}.")
        return

    print(f"\n--- Rozpoczynanie analizy {method.upper()} dla {n_topics} tematów ---")
    start_time = time.time()
    model = None # Inicjalizacja zmiennej model

    try:
        # 1. Trenowanie modelu
        print("Trenowanie modelu...")
        model_train_time = time.time()
        if method == 'lsa':
            # LSI (LSA) Model - używa korpusu TF-IDF
            # Upewnij się, że n_topics nie jest zbyt duże (max to liczba cech w słowniku)
            effective_n_topics = min(n_topics, len(dictionary))
            if effective_n_topics < n_topics:
                 print(f"  Ostrzeżenie: Zmniejszono liczbę tematów z {n_topics} do {effective_n_topics} (maks. rozmiar słownika).")
            if effective_n_topics <= 0:
                 print(f"  BŁĄD: Nie można wytrenować modelu LSA z {effective_n_topics} tematami.")
                 return

            model = LsiModel(
                corpus=corpus, # Użyj corpus_tfidf przekazanego do funkcji
                id2word=dictionary,
                num_topics=effective_n_topics
            )
        elif method == 'lda':
            # LDA Model - używa korpusu BoW
            # LdaMulticore jest szybsze, jeśli dostępne są rdzenie
            # Użyj workers=1 dla stabilności w Colab (unikanie problemów z pamięcią/dyskiem)
            effective_n_topics = n_topics
            if effective_n_topics <= 0:
                 print(f"  BŁĄD: Nie można wytrenować modelu LDA z {effective_n_topics} tematami.")
                 return

            model = LdaMulticore(
                corpus=corpus, # Użyj corpus_bow przekazanego do funkcji
                id2word=dictionary,
                num_topics=effective_n_topics,
                random_state=42,
                chunksize=100,      # Liczba dokumentów przetwarzanych w jednej iteracji
                passes=10,          # Liczba przebiegów przez korpus
                iterations=50,      # Maksymalna liczba iteracji w ramach przebiegu (dla zbieżności)
                workers=6           # WAŻNE DLA COLAB: Użyj 1 rdzenia, aby uniknąć błędów pamięci/dysku
                # per_word_topics=True # Potrzebne do niektórych zaawansowanych analiz, ale zużywa więcej pamięci
            )
        else:
            raise ValueError("Nieprawidłowa metoda. Wybierz 'lsa' lub 'lda'.")
        print(f"Model wytrenowany. Czas trenowania: {time.time() - model_train_time:.2f}s")

        # 2. Obliczanie spójności C_v
        print("Obliczanie spójności C_v...")
        coherence_time = time.time()
        coherence_model = CoherenceModel(
            model=model,
            texts=processed_docs, # WAŻNE: Potrzebuje oryginalnych tokenizowanych tekstów
            dictionary=dictionary,
            coherence='c_v'
        )
        coherence_score = coherence_model.get_coherence()
        print(f"Spójność C_v: {coherence_score:.4f}")
        print(f"Czas obliczania spójności: {time.time() - coherence_time:.2f}s")

        # 3. Ekstrakcja i zapis wyników
        print("Ekstrakcja tematów i zapisywanie wyników...")
        results_time = time.time()
        # Pobierz tematy w wymaganym formacie
        # formatted=False zwraca listę krotek (słowo, wynik)
        topics = model.show_topics(num_topics=-1, num_words=top_n_words, formatted=False)

        topic_results = []
        for topic_id, word_score_pairs in topics:
            words = [word for word, score in word_score_pairs]
            scores = [score for word, score in word_score_pairs]
            topic_results.append({
                'topic_id': topic_id,
                'words': str(words),   # Konwersja listy słów na string
                'scores': str(scores)  # Konwersja listy wyników na string
            })

        # Utwórz DataFrame i zapisz do CSV
        results_df = pd.DataFrame(topic_results)
        output_filename = os.path.join(output_dir, f"{method}_{n_topics}_topics_gensim.csv") # Dodano _gensim do nazwy
        results_df.to_csv(output_filename, index=False, encoding='utf-8')

        print(f"Wyniki zapisane do pliku: {output_filename}")
        print(f"Czas ekstrakcji i zapisu: {time.time() - results_time:.2f}s")

    except Exception as e:
        print(f"Wystąpił błąd podczas analizy {method.upper()} dla {n_topics} tematów: {e}")
        import traceback
        traceback.print_exc() # Wydrukuj pełny traceback błędu

    finally:
        # Zawsze próbuj zwolnić pamięć
        print(f"--- Analiza {method.upper()} dla {n_topics} tematów zakończona. Całkowity czas: {time.time() - start_time:.2f}s ---")
        del model # Usuń model, aby zwolnić pamięć
        if 'coherence_model' in locals():
            del coherence_model
        if 'results_df' in locals():
            del results_df
        if 'topic_results' in locals():
            del topic_results
        gc.collect()

In [5]:
# @title Komórka 5: Uruchomienie analizy Gensim dla LSA i LDA

# Sprawdź, czy potrzebne zmienne istnieją i nie są puste
if ('processed_docs' in locals() and processed_docs and
    'dictionary' in locals() and dictionary and
    'corpus_bow' in locals() and corpus_bow and
    'corpus_tfidf' in locals() and corpus_tfidf):

    # --- Uruchomienie dla LSA (używa corpus_tfidf) ---
    print("\n=== Rozpoczynanie analizy LSA (Gensim) ===")
    for n_topics in NUM_TOPICS_LIST:
        run_gensim_topic_modeling(processed_docs, dictionary, corpus_tfidf, 'lsa', n_topics, TOP_N_WORDS, OUTPUT_DIR)
        gc.collect()

    # --- Uruchomienie dla LDA (używa corpus_bow) ---
    print("\n=== Rozpoczynanie analizy LDA (Gensim) ===")
    for n_topics in NUM_TOPICS_LIST:
        run_gensim_topic_modeling(processed_docs, dictionary, corpus_bow, 'lda', n_topics, TOP_N_WORDS, OUTPUT_DIR)
        gc.collect()

    print("\nWszystkie analizy Gensim zakończone.")
    print(f"Wyniki znajdują się w katalogu: {OUTPUT_DIR}")

    # Wyświetlenie listy plików w katalogu wyników
    print("\nWygenerowane pliki:")
    try:
        for filename in os.listdir(OUTPUT_DIR):
            if filename.endswith(".csv"):
                print(f"- {filename}")
    except FileNotFoundError:
        print(f"Katalog '{OUTPUT_DIR}' nie został znaleziony.")

else:
    print("Nie można uruchomić modelowania tematów Gensim.")
    print("Sprawdź, czy poprzednie komórki wykonały się poprawnie i czy zmienne")
    print("'processed_docs', 'dictionary', 'corpus_bow', 'corpus_tfidf' zostały utworzone.")

# Opcjonalnie: Spakuj wyniki
# import shutil
# try:
#     shutil.make_archive('topic_modeling_results_gensim', 'zip', OUTPUT_DIR)
#     print("\nWyniki Gensim spakowane do pliku topic_modeling_results_gensim.zip")
# except Exception as e:
#     print(f"Błąd podczas pakowania wyników: {e}")


=== Rozpoczynanie analizy LSA (Gensim) ===

--- Rozpoczynanie analizy LSA dla 75 tematów ---
Trenowanie modelu...
Model wytrenowany. Czas trenowania: 46.06s
Obliczanie spójności C_v...
Spójność C_v: 0.3024
Czas obliczania spójności: 61.83s
Ekstrakcja tematów i zapisywanie wyników...
Wyniki zapisane do pliku: ../../results/topic_modeling_results_gensim/lsa_75_topics_gensim.csv
Czas ekstrakcji i zapisu: 0.04s
--- Analiza LSA dla 75 tematów zakończona. Całkowity czas: 107.94s ---

--- Rozpoczynanie analizy LSA dla 100 tematów ---
Trenowanie modelu...
Model wytrenowany. Czas trenowania: 50.46s
Obliczanie spójności C_v...
Spójność C_v: 0.2978
Czas obliczania spójności: 77.23s
Ekstrakcja tematów i zapisywanie wyników...
Wyniki zapisane do pliku: ../../results/topic_modeling_results_gensim/lsa_100_topics_gensim.csv
Czas ekstrakcji i zapisu: 0.03s
--- Analiza LSA dla 100 tematów zakończona. Całkowity czas: 127.73s ---

=== Rozpoczynanie analizy LDA (Gensim) ===

--- Rozpoczynanie analizy LDA 